# This is the Function used for attack (C&W). 

In [ ]:
# CW Attack
from torch import nn
import torch
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
def cw_l2_attack(model, images, labels, targeted=False, c=1e-4, kappa=0, max_iter=1000, learning_rate=0.01) :
    images = images.to(device)     
    labels = labels.to(device)
    # Define f-function
    def f(x) :
        outputs = model(x)
        one_hot_labels = torch.eye(len(outputs[0]))[labels].to(device)
        i, _ = torch.max((1-one_hot_labels)*outputs, dim=1)
        j = torch.masked_select(outputs, one_hot_labels.byte())       
        # If targeted, optimize for making the other class most likely 
        if targeted :
            return torch.clamp(i-j, min=-kappa)       
        # If untargeted, optimize for making the other class most likely 
        else :
            return torch.clamp(j-i, min=-kappa)
    w = torch.zeros_like(images, requires_grad=True).to(device)
    optimizer = torch.optim.Adam([w], lr=learning_rate)
    prev = 1e10
    for step in range(max_iter) :
        a = 1/2*(nn.Tanh()(w) + 1)
        loss1 = nn.MSELoss(reduction='sum')(a, images)
        loss2 = torch.sum(c*f(a))
        cost = loss1 + loss2
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        # Early Stop when loss does not converge.
        if step % (max_iter//10) == 0 :
            if cost > prev :
                print('Attack Stopped due to CONVERGENCE....')
                return a
            prev = cost
        #print(f'Learning Progress : {(step+1)/max_iter*100}')
    attack_images = 1/2*(nn.Tanh()(w) + 1)
    return attack_images

# This is how it should be used 

In [ ]:
print("Attack Image & Predicted Label")
model.eval()
correct = 0
total = 0
for images, labels in testloader:
    images = cw_l2_attack(model, images, labels, targeted=False, c=0.1)
    labels = labels.to(device)
    outputs = model(images)
    _, pre = torch.max(outputs.data, 1)
    total += batch_size
    correct += (pre == labels).sum()
    #plt.imshow(torchvision.utils.make_grid(images.cpu().data, normalize=True), [normal_data.classes[i] for i in pre])
print('Accuracy of test set: %f %%' % (100 * float(correct) / total))

# PGD Attack based on FGSM

In [ ]:
# PGD Attack
def pgd_attack(model, images, labels, alpha = 8/255, eps = 0.1, minimum = 0, maximum = 1, iterations = 100) :
    images = images.to(device)
    labels = labels.to(device)
    loss = nn.CrossEntropyLoss()
        
    original_images = images.data
        
    for i in range(iterations) :    
        images.requires_grad = True
        outputs = model(images)
        model.zero_grad()
        cost = loss(outputs, labels).to(device)
        cost.backward()
        adv_images = images + alpha*images.grad.sign()
        eta = torch.clamp(adv_images - original_images, min=-eps, max=eps)
        images = torch.clamp(original_images + eta, min=minimum, max=maximum).detach_()
    return images

# Performing the attack

In [ ]:
model.eval()

correct = 0
total = 0
#You can put train_loader as well
for images, labels in test_loader:
    alpha     = 8/255  #learning rate step in the direction of the attack
    epsilon   = 0.1    #Upper_Bound on the infinity norm of the perturbation. \|\delta\|_{infty} < epsilon 
    minimum   = 0      #Lower bound of the input value, i.e. 0 in the case if we want the image between [0,1]
    maximum   = 1      #Upper bound of the input value, i.e  1 in the case if we want the image between [0,1]
    iteration = 100    #Number of iterations performed to achieve the attack
    
    images = pgd_attack(model, images, labels)
    labels = labels.to(device)
    outputs = model(images)
    
    _, pre = torch.max(outputs.data, 1)

    total += batch_size
    correct += (pre == labels).sum()
        
print(f'Accuracy of test text: {(100 * float(correct) / total)}')